In [1]:
import gdspy
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy


In [2]:
# The GDSII file is called a library, which contains multiple cells.
lib = gdspy.GdsLibrary(unit=1e-6, precision=1e-9)

# Geometry must be placed in cells.
cell = lib.new_cell('anisotropy-assisted_QPM')

tol = 0.0001 # This control how good circles/arcs really is
fontsize = 10


In [3]:
#directory = "C:\\Users\\conde\\Desktop"
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\Anisotropy-Assisted QPM\\Zcut_t600nm_w750nm_1550TE0-to-775TE1"
files_raw = os.listdir(directory)
files = []

# initialize counter
counter = 0

# iterate through files and check if file has extension .csv, .dat, or .txt
for file in files_raw:
    if file.endswith(".csv") or file.endswith(".dat") or file.endswith(".txt"):
        print(f"{counter}: {file}")
        files.append(file)
        counter += 1
        

0: a1_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1300000rad-m_w=750nm_Lambda=4.83322um_ds=25nm_a=0.05.dat
1: a2_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1300000rad-m_w=750nm_Lambda=4.83322um_ds=25nm_a=0.1.dat
2: a3_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1300000rad-m_w=750nm_Lambda=4.83322um_ds=25nm_a=0.15.dat
3: a4_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1300000rad-m_w=750nm_Lambda=4.83322um_ds=25nm_a=0.2.dat
4: a5_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1300000rad-m_w=750nm_Lambda=4.83322um_ds=25nm_a=0.25.dat
5: b1_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1325000rad-m_w=750nm_Lambda=4.74203um_ds=25nm_a=0.05.dat
6: b2_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1325000rad-m_w=750nm_Lambda=4.74203um_ds=25nm_a=0.1.dat
7: b3_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1325000rad-m_w=750nm_Lambda=4.74203um_ds=25nm_a=0.15.dat
8: b4_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1325000rad-m_w=750nm_Lambda=4.74203um_ds=25nm_a=0.2.dat
9: b5_Anisotropy-Assisted-QPM_Z-cut_t400nm_dk=1325000rad-m_w=750nm_Lambda=4.7

In [4]:
# waveguide params
t_waveguide = 0.6
w_waveguide = 0.75
r_waveguide = 100
L_total = 3000
N = 1
#-------------

# doses and names
sample_number = "CCR_AnisotropyQPM_004"
ebeam_brand = "EL6"
ebeam_energy = 50 # in keV
ebeam_dose = "4x90=360" # in uC/cm2
ebeam_current = 2 # in nA
field_size = 1000
#-------------

# aux inputs
l_square = 10.0
w_taper = 0.30
#-------------

# cells offsets
Nx_offset = 0
Ny_offset = 80

x_global_offset = 0
y_global_offset = 100
#-------------

# header offsets
dx_header = -200
dy_header = 25
#-------------


In [5]:
df_delimiter = '\t'
df_header = 0
header_layer = 200

for j in range(N):
    for i in range(len(files_raw)):
        df = pd.read_csv(directory+"\\"+files[i], delimiter=df_delimiter, header=df_header)
        a = float(files_raw[i].split("_")[-1].split("=")[1][:-4])
        Λ = float(files_raw[i].split("_")[-3].split("=")[1][:-2])
        
        X0, Y0 = df[df.columns[0]].values, df[df.columns[1]].values
        X0diff, Y0diff = np.max(X0)-np.min(X0), np.max(Y0)-np.min(Y0)
        X, Y = X0, Y0
        if a==1:
            L_eff = L_total+150
        else:
            L_eff = L_total
        for k in range(int(L_eff/Λ)):
            X = np.concatenate((X, (X0 + (k+1)*X0diff)[1:]), axis=0)
            Y = np.concatenate((Y, (Y0 + (k+1)*Y0diff)[1:]), axis=0)
        X = np.sort(X)
        Y = np.sort(Y)
        curve = np.transpose(np.array([X,Y]))*10**6

        Xfinal = curve[-1][0]
        Yfinal = curve[-1][1]

        dXfinal = (curve[-1]-curve[-2])[0]
        dYfinal = (curve[-1]-curve[-2])[1]
        θfinal = np.arctan(dYfinal/dXfinal)
        
        waveguide = gdspy.FlexPath(curve, w_waveguide, gdsii_path=True)
        waveguide.turn(r_waveguide, -θfinal)
        waveguide.segment((5400, Yfinal + r_waveguide*(1-np.cos(θfinal))))
        waveguide.translate(i*Nx_offset+x_global_offset, i*Ny_offset+y_global_offset+(i//5)*100)
        cell.add(waveguide)

        waveguide_bottom = gdspy.FlexPath(np.flip(curve[:2], axis=0), w_waveguide, gdsii_path=True)
        waveguide_bottom.turn(r_waveguide, -θfinal)
        waveguide_bottom.segment((-2900, -r_waveguide*(1-np.cos(θfinal))))
        waveguide_bottom.translate(i*Nx_offset+x_global_offset, i*Ny_offset+y_global_offset+(i//5)*100)
        cell.add(waveguide_bottom)

        header1 = "a:"+str(round(a, 3))+"   "
        header2 = "p:"+str(round(Λ, 3))
        header = gdspy.Text(header1+header2, fontsize, layer=header_layer)
        header.translate(dx_header+x_global_offset, i*Ny_offset+dy_header+y_global_offset+(i//5)*100)
        cell.add(header)

header1 = sample_number+"\n"
header2 = "Brand:"+ebeam_brand+"\n"
header3 = "Energy:"+str(ebeam_energy)+"keV \n"
header4 = "Current:"+str(ebeam_current)+"nA \n"
header5 = "Dose:"+str(ebeam_dose)+"uC/cm2 \n\n"
header6 = "w:"+str(round(w_waveguide, 3))+"\n"
header7 = "t:"+str(round(t_waveguide, 3))+"\n"
header8 = "L:"+str(round(L_total, 3))
header_all = header1+header2+header3+header4+header5+header6+header7+header8
header = gdspy.Text(header_all, fontsize, layer=header_layer)
header.translate(-500+x_global_offset,4600+y_global_offset)
cell.add(header)

# axes
material_axis = gdspy.Path(3, (-250+x_global_offset, 4500+100+y_global_offset))
material_axis.segment(50, "-y", layer=header_layer)
material_axis.arc(3, -np.pi, -np.pi/2, layer=header_layer)
material_axis.segment(50, "+x", layer=header_layer)
cell.add(material_axis)

material_X_label = gdspy.Text("X", fontsize, (50-250+x_global_offset, 4500+50+y_global_offset), layer=header_layer)
cell.add(material_X_label)

material_Y_label = gdspy.Text("Y", fontsize, (0-250+x_global_offset, 4500+100+y_global_offset), layer=header_layer)
cell.add(material_Y_label)

brand = gdspy.Text("Z-cut", fontsize, (50-250+x_global_offset, 4500+100+y_global_offset), layer=header_layer)
cell.add(brand)



In [6]:
for i in range(9):
    for j in range(6):
        field_cell = gdspy.Rectangle((0, 0), (field_size, field_size), layer=255)
        field_cell.translate(i*field_size-3000, j*field_size)
        cell.add(field_cell)
        

In [7]:
# Save the library in a file called 'first.gds'.
lib.write_gds(sample_number+'.gds')


C:\Users\conde\AppData\Local\Temp\ipykernel_13920\173651073.py:2: UserWarning: [GDSPY] Paths with more than 8191 are not supported by the official GDSII specification.  This GDSII file might not be compatible with all readers.
  lib.write_gds(sample_number+'.gds')
